In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [17]:
class TrafficSimulation:

    def __init__(self, length=100, car_density=0.2, slow_down_prob=0.5, max_speed=5):
        '''
        Create a new traffic simulation object. Cars are distributed randomly
        along the road and start with random velocities.

        Inputs:

            length (int) The number of cells in the road. Default: 100.

            car_density (float) The fraction of cells that have a car on them.
              Default: 0.2.

            slow_down_prob (float) The probability that a car will randomly
              slow down by 1 during an update step. Default: 0.5.

            max_speed (int) The maximum speed in car cells per update step.
              Default: 5.
        '''
        self.length = length
        self.car_density = car_density
        self.max_speed = max_speed
        self.slow_down_prob = slow_down_prob

        random_indexes_1 = np.random.choice(range(self.length),
                                            size=int(round(car_density * self.length)),
                                            replace=False)
        random_indexes_2 = np.random.choice(range(self.length),
                                            size=int(round(car_density * self.length)),
                                            replace=False)
        
        self.state_1 = -np.ones(self.length, dtype=int)  # -1 means empty cell
        self.state_1[random_indexes_1] = np.random.randint(
            0, self.max_speed + 1, size=len(random_indexes_1))
        
        self.state_2 = -np.ones(self.length, dtype=int)  # -1 means empty cell
        self.state_2[random_indexes_2] = np.random.randint(
            0, self.max_speed + 1, size=len(random_indexes_2))

        # Track the time steps and total number of cars that passed the simulation
        # boundary to estimate average traffic flow.
        self.time_step = 0
        self.cumulative_traffic_flow = 0.

    def initialize(self):
        
    def step(self, display=True):
        '''
        Advance one time step in the simulation.

        Inputs:

            display (bool) Whether to display the state of the simulation. Default: True.
        '''
        # Update car velocities.
        for i in range(self.length):
#             LANE 1
            if self.state_1[i] != -1:
                ahead_same_1 = 1  # The number of empty cells between this car and the next plus 1
                ahead_other_1 = 1
                behind_other_1 = 0
                
                while self.state_1[(i + ahead_same_1) % self.length] == -1:
                    ahead_same_1 += 1
                while self.state_2[(i - ahead_other_1) % self.length] == -1:
                    ahead_other_1 += 1
                while self.state_2[(i - behind_other_1) % self.length] == -1:
                    behind_other_1 += 1
                    
#                 Switch lane
                if behind_other_1 <= 5 and ahead_same_1 < self.state_1[i] + 1:
                    self.state_2[i] = self.state_1[i]
                    self.state_1[i] = -1
#                 On the same lane  
                else:
                # Acceleration
                    if self.state_1[i] < self.max_speed:
                        self.state_1[i] += 1
                    # Deceleration
                    if self.state_1[i] >= ahead_same_1:
                        self.state_1[i] = ahead_same_1 - 1
                    # Randomization
                    if (self.state_1[i] > 0) and (np.random.uniform() < self.slow_down_prob):
                        self.state_1[i] -= 1
                        
#             LANE 2
            if self.state_2[i] != -1:
                ahead_same_2 = 1  # The number of empty cells between this car and the next plus 1
                ahead_other_2 = 1
                behind_other_2 = 0
                
                while self.state_2[(i + ahead_same_2) % self.length] == -1:
                    ahead_same_2 += 1
                while self.state_1[(i - ahead_other_2) % self.length] == -1:
                    ahead_other_2 += 1
                while self.state_1[(i - behind_other_2) % self.length] == -1:
                    behind_other_2 += 1
                    
#                 Switch lane
                if behind_other_2 <= 5 and ahead_same_2 < self.state_2[i] + 1:
                    self.state_1[i] = self.state_2[i]
                    self.state_2[i] = -1
#                 On the same lane  
                else:
                # Acceleration
                    if self.state_2[i] < self.max_speed:
                        self.state_2[i] += 1
                    # Deceleration
                    if self.state_2[i] >= ahead_same_2:
                        self.state_2[i] = ahead_same_2 - 1
                    # Randomization
                    if (self.state_2[i] > 0) and (np.random.uniform() < self.slow_down_prob):
                        self.state_2[i] -= 1


        if display:
            self.display()

        # Move cars forward using their new velocities
        new_state_1 = -np.ones(self.length, dtype=int)
        new_state_2 = -np.ones(self.length, dtype=int)
        for i in range(self.length):
            if self.state_1[i] != -1:
                new_state_1[(i + self.state_1[i]) % self.length] = self.state_1[i]
            if self.state_2[i] != -1:
                new_state_2[(i + self.state_2[i]) % self.length] = self.state_2[i]
        self.state_1 = new_state_1
        self.state_2 = new_state_2

        # Update time and traffic flow
        self.time_step += 1
        for i in range(self.max_speed):
            if self.state_1[i] > i:
                self.cumulative_traffic_flow += 1
            if self.state_2[i] > i:
                self.cumulative_traffic_flow += 1

    def display(self):
        '''
        Print out the current state of the simulation.
        '''
        print(''.join('.' if x == -1 else str(x) for x in self.state_1))
        print(''.join('.' if x == -1 else str(x) for x in self.state_2))

IndentationError: expected an indented block (<ipython-input-17-ef2926e8f2f9>, line 48)

In [18]:
sim = TrafficSimulation()
for i in range(10):
    print('step',i)
    sim.step()
    print('')

step 0
...2....1.2...5...22....5............4..............2..1.00.0.1.041...5........05....4...5..5.......
.001...0.4...........5................1.1..04......................1.4.............1..........1..3..

step 1
3....2...1..3......003.......4...........4............0.001.1..00..0.4.........0.........4....1..5..
.01.1..1.....5............4............0.0.1....5...................1....4.4........15.........2....

step 2
...3...2..2....2...01...4........4...........5........1.01.0.0.01..1.....5...4.1...............1....
.001.1..1.........5...........4........1.1..2........4...............1.........5.....2....5..2...3..

step 3
3........2..3....1.1.1......4........5..................1..1.1.0.1..1.........1.05..5...........2...
.01.1.2..2.............5..........4.....1.1...3...4....1.42...........1................2.......3....

step 4
...4.......3......0.1.1.........5.........5..............1..2..0..1..1.........00.....1..5........3.
.1.1.2..1..2...3............5.........2..0.2.....3..